In [ ]:
import pandas as pd
from chanlun.backtesting import backtest
from chanlun.strategy import strategy_a_3mmd
from chanlun.cl_utils import query_cl_chart_config

## 注意事项
回测前要确保已经同步回测时间段内的行情数据到数据库中

同步行情数据请参考 /script/crontab/ 目录下的脚本

## 注意事项
回测前要确保已经同步回测时间段内的行情数据到数据库中

同步行情数据请参考 /script/crontab/ 目录下的脚本

In [ ]:
# 量化配置
bt_config = {
    # 策略结果保存的文件
    'save_file': './data/bk/currency_strategy_demo.pkl',
    # 设置策略对象
    'strategy': strategy_a_3mmd.StrategyA3mmd(),
    # 回测模式：signal 信号模式，固定金额开仓； trade 交易模式，按照实际金额开仓
    'mode': 'trade',
    # 市场配置，currency 数字货币  a 沪深  hk  港股  futures  期货
    'market': 'currency',
    # 基准代码，用于获取回测的时间列表
    'base_code': 'BCH/USDT',
    # 回测的标的代码
    'codes': ['BCH/USDT', 'EOS/USDT'],
    # 回测的周期，这里设置里，在策略中才能取到对应周期的数据
    'frequencys': ['30m', '5m'],
    # 回测开始的时间
    'start_datetime': '2022-02-01 00:00:00',
    # 回测的结束时间
    'end_datetime': '2022-04-01 00:00:00',
    # 是否是股票，True 当日开仓不可平仓，False 当日开当日可平
    'is_stock': False,
    # 是否是期货，True 可做空，False 不可做空
    'is_futures': True,
     # mode 为 trade 生效，初始账户资金
    'init_balance': 1000000,
    # mode 为 trade 生效，交易手续费率
    'fee_rate': 0.0006,
    # mode 为 trade 生效，最大持仓数量（分仓）
    'max_pos': 3,
    # 缠论计算的配置，详见缠论配置说明
    'cl_config': query_cl_chart_config('currency', 'BTC/USDT'),
}

BT = backtest.BackTest(bt_config)
BT.datas.load_data_to_cache = True # 是否将所有k线数据加载到内存中，如果内存吃紧，可以设置为 False，这样每次循环都会从数据库中读取
BT.datas.load_kline_nums = 10000 # 每次 klines 方法中返回的k线数据条数
BT.datas.cl_data_kline_max_nums = 30000 # 回测过程中，如果缠论对象的数据量超过这个值，则会清空重新计算，cl_data_kline_max_nums 不能小于 load_kline_nums，否则将会不断的删除并重新计算

In [ ]:
# 运行回测
BT.run()
# 多进程回测，只支持信号回测模式
# BT.run_process(max_workers=4)
# 保存回测结果到文件中
BT.save()
print('Done')

In [ ]:
# 输出回测结果
BT.info()
BT.result()

In [ ]:
# 回测数据展示
BT.backtest_charts()

In [ ]:
# 显示历史持仓
#设置显示全部行，不省略
pd.set_option('display.max_rows',None)
#设置显示全部列，不省略
pd.set_option('display.max_columns',None)

# 显示历史持仓
pos_df = BT.positions()
pos_df

In [ ]:
# 展示图表
BT.show_charts(BT.codes[0], BT.frequencys[0])